In [5]:
from sklearn import datasets
import numpy as np
import pandas as pd

In [2]:
import math

In [168]:
iris=datasets.load_iris()
X=iris.data
y=iris.target

In [56]:
def Entropy(y):
    y=np.array(y).reshape(-1,1)
    Counts=pd.DataFrame(y)[0].value_counts()
    keys=Counts.keys().tolist()
    values=Counts.tolist()
    Sum=np.sum(values)
    #print(keys,values)
    plogp=np.sum([-(i/Sum)*math.log2(i/Sum) for i in values if(i!=0)])        
    return plogp

In [57]:
y=[1,1,2,2,3,3,3,2,1,1,2,3,4,1,2]
Entropy(y)

1.8256051990169437

In [58]:
def gain_ratio(y,y_list):
    lengths=[len(i) for i in y_list]
    weights=(lengths)/np.sum(lengths)
    entropy_current=Entropy(y)
    entropies=[Entropy(i) for i in y_list]
    result=np.sum(np.multiply(weights,entropies))
    split_info=np.sum([-(i)*math.log2(i) for i in weights if(i!=0)]) 
    Gain=entropy_current-result
    #print('split_info',split_info)
    if(split_info==0):
        return Gain
    GainRatio=Gain/split_info
    return GainRatio

In [128]:
### f is column index
def gain_fc(X,y,f):
    df_X=pd.DataFrame(X)
    df_y=pd.DataFrame(y)
    f_keys=df_X[f].value_counts().keys().tolist()
    #print(f_keys)
    y_list=[]
    for key in f_keys:
        #print('Key=',key)
        tempX=df_X[df_X[f]==key].values
        tempY=df_y[df_X[f]==key].values
        #print(tempX,tempY)
        y_list.append(tempY)
    #print(y_list)
    if(len(y_list)==1):
        return 0
    ans=gain_ratio(y,y_list)
    return ans,f_keys

In [90]:
gain_fc(X,y,0)

0.31127812445913283

In [135]:
def gain_fn(X,y,f):
    y=y.reshape(-1,1)
    df_X=pd.DataFrame(X)
    df_X=df_X.sort_values([f])
    X=df_X.values
    y=y[np.array(df_X.index)]
    df_y=pd.DataFrame(y)
    
    poss=[]
    for i in range(X.shape[0]-1):
        poss.append((X[i][f]+X[i+1][f])/2)
   
    MaxGain=-9999999
    MaxI=-9999999
    for i in poss:
        y_list=[]
        y_list.append(y[X[:,f]>=i])
        y_list.append(y[X[:,f]<i])
        #print("len 0=",len(y_list[0]),"len 1=",len(y_list[1]))
        gain=gain_ratio(y,y_list)
        #print("i:Gain(i)",i,gain)
        if(gain>MaxGain):
            MaxGain=gain
            MaxI=i
    return MaxI,MaxGain
    

In [163]:
def SelectBestNumerical(X,y,f_list):
    if(len(f_list)==0):
        return -999999,-999999,-999999 
    Bestf=-999999
    BestAns=-999999
    Critical_X=-999999
    for i in f_list:
        #print("y best=",y)
        MaxI,ans=gain_fn(X,y,i)
        if(ans>BestAns):
            BestAns=ans
            Bestf=i
            Critical_X=MaxI
    return BestAns,Bestf,Critical_X


def SelectBestCategorical(X,y,f_list):
    if(len(f_list)==0):
        return -999999,-999999,-999999 
    Bestf=-999999
    BestAns=-999999
    BestKeys=[]
    #print('f=',f)
    for i in f_list:
        ans,keys=gain_fc(X,y,i)
        if(ans>BestAns):
            BestAns=ans
            Bestf=i
            BestKeys=keys
    return BestAns,Bestf,BestKeys


In [186]:
def decision_tree(X,y,f,level):
    df_X=pd.DataFrame(X)
    df_y=pd.DataFrame(y)
    
    print('Level =',level)
    Counts=pd.DataFrame(y)[0].value_counts()
    keys=Counts.keys().tolist()
    values=Counts.values.tolist()
    for i in range(len(keys)):
        print('Count of ',keys[i],' = ',values[i])
    
    print('Current Entropy is=',Entropy(y))
    
    if(df_y.nunique()[0]==1):
        print("Reached Leaf Node")
        return
    
    print("Level",level,"f",f)
    if(len(f)==0):
        print('Features Exhausted')
        return
    
    numerical_f=df_X._get_numeric_data().columns
    if(len(numerical_f)==0):
        numerical_f=[]
    categorical_f=set(f)-set(numerical_f)
    if(len(categorical_f)==0):
        categorical_f=[]
    else:
        categorical_f= np.array(categorical_f)
        
    ans1,f1,Critical_X=SelectBestNumerical(X,y,numerical_f)
    ans2,f2,keys=SelectBestCategorical(X,y,categorical_f)
    
    if(ans1>ans2):
       
        df_X=df_X.sort_values([f1])
        X=df_X.values
        y=y[np.array(df_X.index)]
        df_y=pd.DataFrame(y)
        
        print('Spliting on X[',f1,'] with Gain Ratio=',ans1)
        
        new_f=[i for i in f if(i!=f1)]
        X1=df_X[df_X[f1]<Critical_X].values
        y1=df_y[0][df_X[f1]<Critical_X].values
        print('Level',level,' 1st Part,f=',new_f,'X1.sahpe',X1.shape)
        decision_tree(X1,y1,new_f,level+1)
        
    
        #print(f_list,f1)
        X2=df_X[df_X[f1]>=Critical_X].values
        y2=df_y[df_X[f1]>=Critical_X].values
        print('Level',level,' 2nd Part ,f=',new_f,'X2.sahpe=',X2.shape)
        decision_tree(X2,y2,new_f,level+1)
        
    else:

        print('Spliting on X',f2,' with Gain Ratio=',ans2)
        new_f=[i for i in f if(i!=f2)]
        for key in keys:
            temp_X=df_X[df_X[f2]==key].values
            temp_y=df_y[df_X[f2]==key].values
            decision_tree(temp_X,temp_y,new_f,level+1)
    print('  ')
    
    

In [187]:
decision_tree(X,y,np.arange(X.shape[1]),0)

Level = 0
Count of  2  =  50
Count of  1  =  50
Count of  0  =  50
Current Entropy is= 1.584962500721156
Level 0 f [0 1 2 3]
Spliting on X[ 2 ] with Gain Ratio= 0.9999999999999999
Level 0  1st Part,f= [0, 1, 3] X1.sahpe (50, 4)
Level = 1
Count of  0  =  50
Current Entropy is= 0.0
Reached Leaf Node
Level 0  2nd Part ,f= [0, 1, 3] X2.sahpe= (100, 4)
Level = 1
Count of  2  =  50
Count of  1  =  50
Current Entropy is= 1.0
Level 1 f [0, 1, 3]


/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Spliting on X[ 3 ] with Gain Ratio= 0.6933647985912663
Level 1  1st Part,f= [0, 1] X1.sahpe (54, 4)
Level = 2
Count of  1  =  45
Count of  2  =  9
Current Entropy is= 0.6500224216483541
Level 2 f [0, 1]
Spliting on X[ 2 ] with Gain Ratio= 0.6958832172268378
Level 2  1st Part,f= [0, 1] X1.sahpe (48, 4)
Level = 3
Count of  1  =  42
Count of  2  =  6
Current Entropy is= 0.5435644431995964
Level 3 f [0, 1]
Spliting on X[ 2 ] with Gain Ratio= 0.6293991066688157
Level 3  1st Part,f= [0, 1] X1.sahpe (45, 4)
Level = 4
Count of  1  =  42
Count of  2  =  3
Current Entropy is= 0.35335933502142136
Level 4 f [0, 1]
Spliting on X[ 2 ] with Gain Ratio= 0.11269953191028265
Level 4  1st Part,f= [0, 1] X1.sahpe (29, 4)
Level = 5
Count of  1  =  29
Current Entropy is= 0.0
Reached Leaf Node
Level 4  2nd Part ,f= [0, 1] X2.sahpe= (16, 4)
Level = 5
Count of  1  =  13
Count of  2  =  3
Current Entropy is= 0.6962122601251459
Level 5 f [0, 1]
Spliting on X[ 1 ] with Gain Ratio= 0.12838011484379866
Level 5  1st